<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">



---

## **TLDR**

* **Dashboard**:
Google Data Studio [Link](https://lookerstudio.google.com/reporting/6a5f2a78-fce1-4a4d-9b31-c1e83e3a78f6) .
* **Processamento**:
Kaggle Notebook [Link](https://www.kaggle.com/code/vidafreitas/project-covid-19).
* **Fontes:**
Casos pela universidade John Hopkins (link);
Vacinação pela universidade de Oxford (link).

# **Projeto COVID-19**

## 1\. Contexto

> A COVID-19 é uma infecção respiratória aguda causada pelo coronavírus SARS-CoV-2, potencialmente grave, de elevada transmissibilidade e de distribuição global. Fonte: Governo brasileiro ([link](https://www.gov.br/saude/pt-br/coronavirus/o-que-e-o-coronavirus)).

Neste projeto, a condução da extração, transformação e carregamento dos dados (ETL) tem como propósito a construção de um dashboard para exploração e visualização interativa de informações relacionadas ao avanço de casos e à vacinação no Brasil.

Serão utilizadas duas bases de dados distintas. A primeira base de dados abrange informações sobre a pandemia de COVID-19, incluindo casos, mortes, entre outras estatísticas. Esses dados foram compilados pelo Centro de Ciência de Sistemas e Engenharia da Universidade Americana Johns Hopkins [Link](https://www.jhu.edu/).

A segunda base de dados contém informações sobre a vacinação contra a COVID-19 e foi compilada pelo projeto "Nosso Mundo em Dados" (Our World in Data ou OWID) da Universidade Britânica de Oxford [Link](https://covid.ourworldindata.org/data/owid-covid-data.csv). Esses dados são atualizados diariamente desde janeiro de 2020, proporcionando uma granularidade temporal de dias e uma abrangência geográfica que inclui países.

## 2\. Análise Exploratória de Dados

Neste projeto iremos utilizar os seguintes pacotes Python para processar os dados bruto em um formato adequado para análise.

In [ ]:
import math
from typing import Iterator
from datetime import datetime, timedelta
import numpy as np
import pandas as pd

### 2.1 ETL - COVID-19 *Cases*

**Nesta etapa, vamos realizar a extração, transformação e carregamento do dataset de casos, seguindo os passos abaixo:**

1. Fazer a extraçao dos dados.
2. Selecionar as colunas de interesse do dataset do Brasil.
3. Manipulação dos dados com foco em garantir uma boa granularidade e qualidade da base de dados.
4. Enriquecimento da base de dados.
5. Realizar o type casting e organizar as colunas

In [ ]:
cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-12-2021.csv', sep=',')

In [ ]:
cases.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-01-13 05:22:15,33.93911,67.709953,53584,2301,44608,6675,Afghanistan,137.647787,4.294192
1,NaN,NaN,NaN,Albania,2021-01-13 05:22:15,41.15330,20.168300,64627,1252,38421,24954,Albania,2245.708527,1.937271
2,NaN,NaN,NaN,Algeria,2021-01-13 05:22:15,28.03390,1.659600,102641,2816,69608,30217,Algeria,234.067409,2.743543
3,NaN,NaN,NaN,Andorra,2021-01-13 05:22:15,42.50630,1.521800,8682,86,7930,666,Andorra,11236.653077,0.990555
4,NaN,NaN,NaN,Angola,2021-01-13 05:22:15,-11.20270,17.873900,18343,422,15512,2409,Angola,55.811022,2.300605


In [ ]:
def date_range(start_date: datetime, end_date: datetime) -> Iterator[datetime]:
  date_range_days: int = (end_date - start_date).days
  for lag in range(date_range_days):
    yield start_date + timedelta(lag)


start_date = datetime(2021,  1,  1)
end_date   = datetime(2021, 12, 31)

In [ ]:
cases = None
cases_is_empty = True

for date in date_range(start_date=start_date, end_date=end_date):

  date_str = date.strftime('%m-%d-%Y')
  data_source_url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{date_str}.csv'

  case = pd.read_csv(data_source_url, sep=',')

  case = case.drop(['FIPS', 'Admin2', 'Last_Update', 'Lat', 'Long_', 'Recovered', 'Active', 'Combined_Key', 'Case_Fatality_Ratio'], axis=1)
  case = case.query('Country_Region == "Brazil"').reset_index(drop=True)
  case['Date'] = pd.to_datetime(date.strftime('%Y-%m-%d'))

  if cases_is_empty:
    cases = case
    cases_is_empty = False
  else:
    cases = pd.concat([cases, case], ignore_index=True)

In [ ]:
cases.head()

,Province_State,Country_Region,Confirmed,Deaths,Incident_Rate,Date
0,Acre,Brazil,41689,796,4726.992352,2021-01-01
1,Alagoas,Brazil,105091,2496,3148.928928,2021-01-01
2,Amapa,Brazil,68361,926,8083.066602,2021-01-01
3,Amazonas,Brazil,201574,5295,4863.536793,2021-01-01
4,Bahia,Brazil,494684,9159,3326.039611,2021-01-01


In [ ]:
cases.query('Province_State == "Rio de Janeiro"').head()

,Province_State,Country_Region,Confirmed,Deaths,Incident_Rate,Date
20,Rio de Janeiro,Brazil,435390,25600,2521.815450,2021-01-01
47,Rio de Janeiro,Brazil,435604,25608,2523.054956,2021-01-02
74,Rio de Janeiro,Brazil,435739,25616,2523.836887,2021-01-03
101,Rio de Janeiro,Brazil,439345,25617,2544.723142,2021-01-04
128,Rio de Janeiro,Brazil,443607,25837,2569.409004,2021-01-05


In [ ]:
cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9828 entries, 0 to 9827
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Province_State  9828 non-null   object        
 1   Country_Region  9828 non-null   object        
 2   Confirmed       9828 non-null   int64         
 3   Deaths          9828 non-null   int64         
 4   Incident_Rate   9828 non-null   float64       
 5   Date            9828 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 460.8+ KB


In [ ]:
cases = cases.rename(
  columns={
    'Province_State': 'state',
    'Country_Region': 'country'
  }
)

for col in cases.columns:
  cases = cases.rename(columns={col: col.lower()})

In [ ]:
states_map = {
    'Amapa': 'Amapá',
    'Ceara': 'Ceará',
    'Espirito Santo': 'Espírito Santo',
    'Goias': 'Goiás',
    'Para': 'Pará',
    'Paraiba': 'Paraíba',
    'Parana': 'Paraná',
    'Piaui': 'Piauí',
    'Rondonia': 'Rondônia',
    'Sao Paulo': 'São Paulo'
}

cases['state'] = cases['state'].apply(lambda state: states_map.get(state) if state in states_map.keys() else state)

In [ ]:
cases['month'] = cases['date'].apply(lambda date: date.strftime('%Y-%m'))
cases['year']  = cases['date'].apply(lambda date: date.strftime('%Y'))

In [ ]:
cases['population'] = round(100000 * (cases['confirmed'] / cases['incident_rate']))
cases = cases.drop('incident_rate', axis=1)

In [ ]:
cases.head()

,state,country,confirmed,deaths,date,month,year,population
0,Acre,Brazil,41689,796,2021-01-01,2021-01,2021,881935.0
1,Alagoas,Brazil,105091,2496,2021-01-01,2021-01,2021,3337357.0
2,Amapá,Brazil,68361,926,2021-01-01,2021-01,2021,845731.0
3,Amazonas,Brazil,201574,5295,2021-01-01,2021-01,2021,4144597.0
4,Bahia,Brazil,494684,9159,2021-01-01,2021-01,2021,14873064.0


In [ ]:
# média móvel (7 dias) e estabilidade (14 dias) de casos e mortes por estado:

cases_ = None
cases_is_empty = True

def get_trend(rate: float) -> str:

  if np.isnan(rate):
    return np.NaN

  if rate < 0.85:
    status = 'downward'
  elif rate > 1.15:
    status = 'upward'
  else:
    status = 'stable'

  return status


for state in cases['state'].drop_duplicates():

  cases_per_state = cases.query(f'state == "{state}"').reset_index(drop=True)
  cases_per_state = cases_per_state.sort_values(by=['date'])

  cases_per_state['confirmed_1d'] = cases_per_state['confirmed'].diff(periods=1)
  cases_per_state['confirmed_moving_avg_7d'] = np.ceil(cases_per_state['confirmed_1d'].rolling(window=7).mean())
  cases_per_state['confirmed_moving_avg_7d_rate_14d'] = cases_per_state['confirmed_moving_avg_7d']/cases_per_state['confirmed_moving_avg_7d'].shift(periods=14)
  cases_per_state['confirmed_trend'] = cases_per_state['confirmed_moving_avg_7d_rate_14d'].apply(get_trend)

  cases_per_state['deaths_1d'] = cases_per_state['deaths'].diff(periods=1)
  cases_per_state['deaths_moving_avg_7d'] = np.ceil(cases_per_state['deaths_1d'].rolling(window=7).mean())
  cases_per_state['deaths_moving_avg_7d_rate_14d'] = cases_per_state['deaths_moving_avg_7d']/cases_per_state['deaths_moving_avg_7d'].shift(periods=14)
  cases_per_state['deaths_trend'] = cases_per_state['deaths_moving_avg_7d_rate_14d'].apply(get_trend)

  if cases_is_empty:
    cases_ = cases_per_state
    cases_is_empty = False
  else:
    cases_ = pd.concat([cases_,cases_per_state], ignore_index=True)

cases = cases_
cases_ = None

In [ ]:
# Type casting das colunas.

cases['population'] = cases['population'].astype('Int64')
cases['confirmed_1d'] = cases['confirmed_1d'].astype('Int64')
cases['confirmed_moving_avg_7d'] = cases['confirmed_moving_avg_7d'].astype('Int64')
cases['deaths_1d'] = cases['deaths_1d'].astype('Int64')
cases['deaths_moving_avg_7d'] = cases['deaths_moving_avg_7d'].astype('Int64')

In [ ]:
# Reorganizar as colunas e conferir o resultado final.

cases = cases[['date', 'country', 'state', 'population', 'confirmed', 'confirmed_1d', 'confirmed_moving_avg_7d', 'confirmed_moving_avg_7d_rate_14d', 'confirmed_trend', 'deaths', 'deaths_1d', 'deaths_moving_avg_7d', 'deaths_moving_avg_7d_rate_14d', 'deaths_trend', 'month', 'year']]

cases.head()

,date,country,state,population,confirmed,confirmed_1d,confirmed_moving_avg_7d,confirmed_moving_avg_7d_rate_14d,confirmed_trend,deaths,deaths_1d,deaths_moving_avg_7d,deaths_moving_avg_7d_rate_14d,deaths_trend,month,year
0,2021-01-01,Brazil,Acre,881935,41689,<NA>,<NA>,NaN,NaN,796,<NA>,<NA>,NaN,NaN,2021-01,2021
1,2021-01-02,Brazil,Acre,881935,41941,252,<NA>,NaN,NaN,798,2,<NA>,NaN,NaN,2021-01,2021
2,2021-01-03,Brazil,Acre,881935,42046,105,<NA>,NaN,NaN,802,4,<NA>,NaN,NaN,2021-01,2021
3,2021-01-04,Brazil,Acre,881935,42117,71,<NA>,NaN,NaN,806,4,<NA>,NaN,NaN,2021-01,2021
4,2021-01-05,Brazil,Acre,881935,42170,53,<NA>,NaN,NaN,808,2,<NA>,NaN,NaN,2021-01,2021


In [ ]:
# Carregamento dos dados manipulados.

cases.to_csv('./covid-cases.csv', sep=',', index=False)

### 2.2 ETL - COVID-19 Vaccines

**Nesta etapa, vamos realizar a extração, transformação e carregamento do dataset de vacinas, seguindo os passos abaixo:**

1. Fazer a extraçao dos dados.
2. Selecionar as colunas de interesse do dataset do Brasil.
3. Tratamento de dados
4. Manipulação dos dados com foco em garantir uma boa granularidade e qualidade da base de dados.
5. Enriquecimento da base de dados.
6. Organizar as colunas

In [ ]:
vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3], infer_datetime_format=True)

In [ ]:
vaccines.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-01-03,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-04,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-01-05,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-01-06,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-01-07,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN


In [ ]:
# Selecionar as colunas de interesse e as linhas referentes ao Brasil.

vaccines = vaccines.query('location == "Brazil"').reset_index(drop=True)
vaccines = vaccines[['location', 'population', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'date']]

vaccines.head()

,location,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,date
0,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-03
1,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-04
2,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-05
3,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-06
4,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-07


In [ ]:
vaccines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1414 entries, 0 to 1413
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   location                 1414 non-null   object        
 1   population               1414 non-null   float64       
 2   total_vaccinations       695 non-null    float64       
 3   people_vaccinated        691 non-null    float64       
 4   people_fully_vaccinated  675 non-null    float64       
 5   total_boosters           455 non-null    float64       
 6   date                     1414 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 77.5+ KB


In [ ]:
# Tratar dados faltantes

vaccines = vaccines.fillna(method='ffill')

In [ ]:
# Filtrar os dados com base na coluna date.

vaccines = vaccines[(vaccines['date'] >= '2021-01-01') & (vaccines['date'] <= '2021-12-31')].reset_index(drop=True)

In [ ]:
# Alterar nomes das colunas

vaccines = vaccines.rename(
  columns={
    'location': 'country',
    'total_vaccinations': 'total',
    'people_vaccinated': 'one_shot',
    'people_fully_vaccinated': 'two_shots',
    'total_boosters': 'three_shots',
  }
)

In [ ]:
# Enriquecer o dataset

vaccines['month'] = vaccines['date'].apply(lambda date: date.strftime('%Y-%m'))
vaccines['year']  = vaccines['date'].apply(lambda date: date.strftime('%Y'))

In [ ]:
vaccines['one_shot_perc'] = round(vaccines['one_shot'] / vaccines['population'], 4)
vaccines['two_shots_perc'] = round(vaccines['two_shots'] / vaccines['population'], 4)
vaccines['three_shots_perc'] = round(vaccines['three_shots'] / vaccines['population'], 4)

In [ ]:
vaccines['population'] = vaccines['population'].astype('Int64')
vaccines['total'] = vaccines['total'].astype('Int64')
vaccines['one_shot'] = vaccines['one_shot'].astype('Int64')
vaccines['two_shots'] = vaccines['two_shots'].astype('Int64')
vaccines['three_shots'] = vaccines['three_shots'].astype('Int64')

In [ ]:
# Organizar as Colunas

vaccines = vaccines[['date', 'country', 'population', 'total', 'one_shot', 'one_shot_perc', 'two_shots', 'two_shots_perc', 'three_shots', 'three_shots_perc', 'month', 'year']]

vaccines.head()

,date,country,population,total,one_shot,one_shot_perc,two_shots,two_shots_perc,three_shots,three_shots_perc,month,year
0,2021-01-01,Brazil,215313504,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN,2021-01,2021
1,2021-01-02,Brazil,215313504,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN,2021-01,2021
2,2021-01-03,Brazil,215313504,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN,2021-01,2021
3,2021-01-04,Brazil,215313504,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN,2021-01,2021
4,2021-01-05,Brazil,215313504,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN,2021-01,2021


In [ ]:
# Carregamento do dataset

vaccines.to_csv('./covid-vaccines.csv', sep=',', index=False)

## 2\. Exploração Interativa e Dados

O dashboard irá conter os **KPIs (key performance indicator)** abaixo:

* Casos e mortes nas 24 horas;
* Média móvel (7 dias) de casos e mortes;
* Tendência de casos e mortes;
* Proporção de vacinados com 1ª, 2ª e 3ª doses.

O dashboard de dados contem os seguintes gráficos para a análise exploratória de dados (exploratory data analysis ou EDA) interativa:

* Distribuição da média móvel (7 dias) do números de casos e mortes ao longo do tempo;
* Concentração dos números de casos confirmados e de mortes por estado.
* Distribuição geográfica dos casos medio confirmados de 1 dia por estado.